In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/customer-segmentation/Train.csv')
data.head()

|Variable|	Definition|
|---|---|
|ID	Unique| ID|
|Gender|	Gender of the customer|
|Ever_Married|	Marital status of the customer|
|Age|	Age of the customer|
|Graduated|	Is the customer a graduate?|
|Profession|	Profession of the customer|
|Work_Experience|	Work Experience in years|
|Spending_Score|	Spending score of the customer|
|Family_Size|	Number of family members for the customer (including the customer)|
|Var_1|	Anonymised Category for the customer|
|Segmentation|	(target) Customer Segment of the customer|

In [ ]:
data.info()

# EDA

In [ ]:
plot_data = data.groupby('Segmentation')['Segmentation'].agg(['count']).reset_index()

fig = px.pie(plot_data, values = plot_data['count'], names = plot_data['Segmentation'])

fig.update_traces(textposition = 'inside', textinfo = 'percent + label', hole = 0.5, 
                  marker = dict(colors = ['#2A3132','#336B87'], line = dict(color = 'white', width = 2)))

fig.update_layout(title_text = 'Customer<br>Segmentation', title_x = 0.5, title_y = 0.55, title_font_size = 26, 
                  title_font_family = 'Calibri', title_font_color = 'black', showlegend = False)
                  
fig.show()

In [ ]:
def plot_category(feature, figsize=None):
    A_count = data[data['Segmentation']=='A'].groupby([feature]).size()
    B_count = data[data['Segmentation']=='B'].groupby([feature]).size()
    C_count = data[data['Segmentation']=='C'].groupby([feature]).size()
    D_count = data[data['Segmentation']=='D'].groupby([feature]).size()
    labels = A_count.index

    x = np.arange(len(labels)) # the label locations
    width = 0.7  # the width of the bars

    if figsize:
        fig, ax = plt.subplots(figsize=figsize)
    else:
        fig, ax = plt.subplots()
    rects1 = ax.bar(x-width/3, round(A_count*100/data.groupby([feature]).size(), 2), 
                    width/5, label='A')
    rects2 = ax.bar(x-width/8, round(B_count*100/data.groupby([feature]).size(), 2), 
                    width/5, label='B')
    rects3 = ax.bar(x+width/8, round(C_count*100/data.groupby([feature]).size(), 2), 
                    width/5, label='C')
    rects4 = ax.bar(x+width/3, round(D_count*100/data.groupby([feature]).size(), 2), 
                    width/5, label='D')

    ax.set_ylabel('Count')
    ax.set_title('Based on %s'%feature)
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=80)
    ax.legend(loc=0, bbox_to_anchor=(1, 1));

    ax.bar_label(rects1, padding=1)
    ax.bar_label(rects2, padding=1)
    ax.bar_label(rects3, padding=1)
    ax.bar_label(rects4, padding=1)

    fig.tight_layout()
    plt.show()
    
def plot_numerical(feature, figsize=None):
    fig = plt.figure(figsize=(10,6))

    sns.kdeplot(data[data['Segmentation']=='A'][feature])
    sns.kdeplot(data[data['Segmentation']=='B'][feature])
    sns.kdeplot(data[data['Segmentation']=='C'][feature])
    sns.kdeplot(data[data['Segmentation']=='D'][feature])

    fig.legend(labels=['Segmentation A', 'Segmentation B', 'Segmentation C', 'Segmentation D'])
    plt.title('Based on %s'%feature)
    plt.show()
    
def plot_pie(feature):
    plot_data = data.groupby([feature, 'Segmentation'])[feature].agg({'count'}).reset_index()

    fig = px.sunburst(plot_data, path = [feature, 'Segmentation'], values = 'count', color = feature, 
                      title = 'Affect of %s on Customer Segmentation'%feature, width = 600, height = 600)

    fig.update_layout(plot_bgcolor = 'white', title_font_family = 'Calibri Black', title_font_color = '#221f1f', 
                      title_font_size = 22, title_x = 0.5)

    fig.update_traces(textinfo = 'label + percent parent')
    fig.show()

In [ ]:
for feature in ['Gender', 'Ever_Married', 'Graduated', 'Spending_Score']:
    plot_pie(feature)

In [ ]:
for feature in ['Profession', 'Var_1']:
    plot_pie(feature)

In [ ]:
for feature in ['Age', 'Work_Experience', 'Family_Size']:
    plot_numerical(feature)

**Observations-**
* Ever_Married - UnMarried customers are usually in segment D while married are in segment A, B or C
* Graduated - Graduated customers are usually in segment A, B or C while Ungraduated are in segment D
* Profession - Customers in healthcare & marketing are mostly in segment D, Artist & engineers are usually in A, B or C
* Spending_Score - Usually 'Low' spenders are in segment A or D while 'high' and 'average' spenders are in segment B or C
* Age - <30 are in segment D, 30-40 or >70 are in segment A while 45-70 are in segment C
* Work_Experience - <2 are in segment C while 6-11 are in segment A & D
* Family_Size - <1 are in segment A, 1-3 are in Segment C and 4+ in segment D

In [ ]:
categorical_features = ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']
numerical_features = ['Age', 'Work_Experience', 'Family_Size']

to_drop = ['ID'] # contain unique values

# CORRELATION

### Label encoding category features for correlation

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import os
import joblib

In [ ]:
df = data.copy()
path = '/kaggle/working'
for i, feature in enumerate(categorical_features):
    le = LabelEncoder()

    # create directory to save label encoding models
    if not os.path.exists(os.path.join(path, "TextEncoding")):
        os.makedirs(os.path.join(path, "TextEncoding"))

    # perform label encoding
    le.fit(df[feature])
    
    # save the encoder
    joblib.dump(le, open(os.path.join(path, "TextEncoding/le_{}.sav".format(feature)), 'wb'))
    
    # transfrom training data
    df[feature] = le.transform(df[feature])

    # get classes & remove first column to elude from dummy variable trap
    columns = list(map(lambda x: feature+' '+str(x), list(le.classes_)))[1:]
    
    # save classes
    joblib.dump(columns, 
                open(os.path.join(path, "TextEncoding/le_{}_classes.sav".format(feature)), 'wb'))

## Bivariate Analysis Correlation plot with the Numeric variables


In [ ]:
plt.figure(figsize=(5, 2))
sns.heatmap(round(data[numerical_features].corr(), 2), annot=True,
            mask=None, cmap='GnBu')
corr_mat = data[numerical_features].corr()
plt.show()

## Bivariate Analysis Correlation plot with the Categorical variables

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(round(df[categorical_features+numerical_features].corr(method='spearman'), 2), annot=True,
            mask=None, cmap='GnBu')
plt.show()

**Observations-**
* Ever_Married - Spending_Score
* Ever_Married - Age

# Analyzing features using VIF

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# Calculating VIF
vif = pd.DataFrame()
temp = df.dropna()
vif["variables"] = [feature for feature in categorical_features+numerical_features if feature not in ['Age', 'Var_1']]
vif["VIF"] = [variance_inflation_factor(temp[vif['variables']].values, i) for i in range(len(vif["variables"]))]
print(vif)

Age and Var_1 have high VIF score

# Handling Missing Values

In [ ]:
missingValueFeatures = pd.DataFrame({'missing %': data.isnull().sum()*100/len(data)})
missingValueFeatures[missingValueFeatures['missing %']>0]

As most of the features are uncorrelated, it is difficult to fill the NA values. Hence for the time being let's drop all NA.

In [ ]:
data_missing = data.dropna().reset_index(drop=True)

# Looking at Outliers

In [ ]:
NumericData = data_missing[[feature for feature in numerical_features if feature not in []]]
NumericMelt = NumericData.melt()
plt.figure(figsize=(15,10))
plt.title("Boxplots for Numerical variables")
bp = sns.boxplot(x='variable', y='value', data=NumericMelt)
bp.set_xticklabels(bp.get_xticklabels(), rotation=90)
plt.show()

In [ ]:
# Percentage of outliers present in each variable
outlier_percentage = {}
for feature in numerical_features:
    tempData = data_missing.sort_values(by=feature)[feature]
    Q1, Q3 = tempData.quantile([0.25, 0.75])
    IQR = Q3 - Q1
    Lower_range = Q1 - (1.5 * IQR)
    Upper_range = Q3 + (1.5 * IQR)
    outlier_percentage[feature] = round((((tempData<(Q1 - 1.5 * IQR)) | (tempData>(Q3 + 1.5 * IQR))).sum()/tempData.shape[0])*100,2)
outlier_percentage

# Handling Categorical Features (Label and One Hot Encoding)

In [ ]:
df = data_missing.copy()
path = '/kaggle/working'
for i, feature in enumerate(categorical_features):
    
    le = LabelEncoder()
    ohe = OneHotEncoder(sparse=False)

    # create directory to save label encoding models
    if not os.path.exists(os.path.join(path, "TextEncoding")):
        os.makedirs(os.path.join(path, "TextEncoding"))

    # perform label encoding
    le.fit(df[feature])
    # save the encoder
    joblib.dump(le, open(os.path.join(path, "TextEncoding/le_{}.sav".format(feature)), 'wb'))
    
    # transfrom training data
    df[feature] = le.transform(df[feature])

    # get classes & remove first column to elude from dummy variable trap
    columns = list(map(lambda x: feature+' '+str(x), list(le.classes_)))[1:]
    
    # save classes
    joblib.dump(columns, 
                open(os.path.join(path, "TextEncoding/le_{}_classes.sav".format(feature)), 'wb'))
    # load classes
    columns = joblib.load(
        open(os.path.join(path, "TextEncoding/le_{}_classes.sav".format(feature)), 'rb'))

    if len(le.classes_)>2:
        # perform one hot encoding
        ohe.fit(df[[feature]])
        # save the encoder
        joblib.dump(ohe, open(os.path.join(path, "TextEncoding/ohe_{}.sav".format(feature)), 'wb'))

        # transfrom training data
        # removing first column of encoded data to elude from dummy variable trap
        tempData = ohe.transform(df[[feature]])[:, 1:]

        # create Dataframe with columns as classes
        tempData = pd.DataFrame(tempData, columns=columns)
    else:
        tempData = df[[feature]]
    
    # create dataframe with all the label encoded categorical features along with hot encoding
    if i==0:
        encodedData = pd.DataFrame(data=tempData, columns=tempData.columns.values.tolist())
    else:
        encodedData = pd.concat([encodedData, tempData], axis=1)

In [ ]:
# merge numerical features and categorical encoded features
df = df[numerical_features+['Segmentation']]
df = pd.concat([df, encodedData], axis=1)
df.info()

# Training Model

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from xgboost import XGBClassifier

In [ ]:
train_data = df.copy()
feature_cols = [feature for feature in train_data.columns if feature not in(['Segmentation'])]
print('features used- ', feature_cols)

''' Rescaling to [0,1] '''
scaler = StandardScaler()
scaler.fit(train_data[feature_cols])
train_data[feature_cols] = scaler.transform(train_data[feature_cols])

In [ ]:
train_data['Segmentation'] = train_data['Segmentation'].map({'A':0, 'B':1, 'C':2, 'D':3})
X = train_data[feature_cols]
y = train_data['Segmentation']

validation_size = 0.25
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=validation_size, 
                                                    random_state=0, stratify=y)

# Model 1: Logistic Regression

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_train)

print('Train metrics...')
print(confusion_matrix(y_train, y_pred))
print(classification_report(y_train, y_pred))

y_pred = model.predict(X_test)

print('Validation metrics...')
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
''' metrics on original data '''
y_pred = model.predict(train_data[feature_cols])

def make_cm(matrix, columns):
    n = len(columns)
    act = ['actual Segmentation'] * n
    pred = ['predicted Segmentation'] * n

    cm = pd.DataFrame(matrix, 
        columns=[pred, columns], index=[act, columns])
    return cm

df_matrix=make_cm(
    confusion_matrix(train_data['Segmentation'], y_pred),['A', 'B', 'C', 'D'])

display(df_matrix)
print(classification_report(train_data['Segmentation'], y_pred))

# Model 2: XGB

In [ ]:
model = XGBClassifier(
    learning_rate=0.05, 
    max_depth=3,
    min_child_weight=5, 
    n_estimators=1000, 
    random_state=7, 
    reg_lambda=1.5,
    reg_alpha=0.5,
    use_label_encoder=False
)

model.fit(X_train, y_train,
          eval_metric='mlogloss',
          verbose=False)

In [ ]:
y_pred = model.predict(X_train)

print('Train metrics...')
print(confusion_matrix(y_train, y_pred))
print(classification_report(y_train, y_pred))

y_pred = model.predict(X_test)

print('Test metrics...')
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
''' metrics on original data '''
y_pred = model.predict(train_data[feature_cols])

def make_cm(matrix, columns):
    n = len(columns)
    act = ['actual Segmentation'] * n
    pred = ['predicted Segmentation'] * n

    cm = pd.DataFrame(matrix, 
        columns=[pred, columns], index=[act, columns])
    return cm

df_matrix=make_cm(
    confusion_matrix(train_data['Segmentation'], y_pred),['A', 'B', 'C', 'D'])

display(df_matrix)
print(classification_report(train_data['Segmentation'], y_pred))

# Model 3: Deep Learning

In [ ]:

np.random.seed(123)  # for reproducibility

import keras
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout

In [ ]:
# Initialize the constructor
model = Sequential()
# Add an input layer 
# arguemtns of dense: output shape, activation, input shape
# activation(define the output function) = [relu', 'tanh']
model.add(Dense(64, activation='relu', input_shape=(len(feature_cols),)))
model.add(Dropout(0.3))
# Add one hidden layer 
# after first layer no need to give input shape
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(8, activation='relu'))
# Add an output layer 
# activation = [regression: 'linear', binary classification: 'sigmoid', multiclass: 'softmax']
# threshold: >0 or <0
# sigmoid: 1/(1+e^-x), usually applied in output layer
# rectifier: max(x,0), usually applied in hidden layer: relu
# hyperbolic tangent tanh: (1-e^-2x)/(1+e^-2x)
model.add(Dense(4, activation='softmax'))

# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
# loss = [regression: 'mse', binary: 'binary_crossentropy', multi: 'categorical_crossentropy']
model.compile(loss='categorical_crossentropy',
              optimizer='RMSprop',
              metrics=['accuracy']
         )

print(model.summary())

In [ ]:
fit = model.fit(X_train, pd.get_dummies(y_train), epochs=150, batch_size=500, verbose=1, validation_split=0.2)

In [ ]:
# Final evaluation of the model
loss, accuracy = model.evaluate(X_test, pd.get_dummies(y_test), verbose=1)
#print("Accuracy: ", accuracy*100, "%")

# plot training vs validation for overfiiting
plt.plot(fit.history['accuracy'], label='train accuracy')
plt.plot(fit.history['val_accuracy'], label='validation accuracy')
plt.plot(fit.history['loss'], label='train loss')
plt.plot(fit.history['val_loss'], label='validation loss')
plt.title('model_accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc=0, bbox_to_anchor=[1,1])
plt.show()